### Libraries

In [0]:
from datetime import datetime

### Configure environment

In [0]:
# Datalake connection variables
storage_account_name    = "dlptde"
container_name          = "data-pt-de"
sas_token               = "sp=racwdle&st=2024-04-24T20:31:21Z&se=2024-05-04T04:31:21Z&spr=https&sv=2022-11-02&sr=c&sig=8g%2B97QOqHn9A6UebuEV0PYbJnCBfZSg2fim%2FsprqfCU%3D"

# Apply settings
spark.conf.set(
    f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net",
    sas_token
)

# SQL connection variables
server_name     = "srvsql-pt-de"
port            = "1433"
database_name   = "sqldb-pt-de"
user_name       = "read_write_user"
password        = "-C2:!*Hcl4O&4e7"
schema_db       = "dbo"
jdbc_url        = f"jdbc:sqlserver://{server_name}.database.windows.net:{port};database={database_name}"

database_properties = {
    "user": user_name,
    "password": password,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

### Source routes

In [0]:
# Save destination paths
comun_path                  = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net"

categories_destination_path = f"{comun_path}/categories_data.csv"
levels_destination_path     = f"{comun_path}/levels_data.csv"
courses_destination_path    = f"{comun_path}/courses.parquet"

date_filter                 = datetime.now() # This variable can be parameterized and received as an input from the notebook
date_filter_str             = date_filter.strftime("%Y-%m-%d %H:%M:%S")
date_filter_str

'2024-04-27 02:04:52'

### Read and filter data

In [0]:
# Read, filter data and create dataframes
df_categories     = spark.read.csv(categories_destination_path, header=True)
df_levels         = spark.read.csv(levels_destination_path, header=True)
df_courses        = spark.read.parquet(courses_destination_path)
df_courses_filter = df_courses.filter(df_courses.start_date >= date_filter_str)

### Write into azure SQL

In [0]:
# Write into sql(DTU)

df_categories.write.jdbc(
    url=jdbc_url, 
    table="dbo.categories", 
    mode="overwrite", 
    properties=database_properties
    )

df_levels.write.jdbc(
    url=jdbc_url, 
    table="dbo.levels", 
    mode="overwrite", 
    properties=database_properties
    )

df_courses_filter.write.jdbc(
    url=jdbc_url, 
    table="dbo.courses", 
    mode="append", 
    properties=database_properties
    )